# Test de generación de ejercicios (LoRA)

Este notebook prueba el endpoint FastAPI `/generate` y valida que la respuesta sea JSON con el schema esperado.


In [ ]:
import json
import requests

API_URL = 'http://localhost:8001/generate'

payload = {
    'topic': 'pandas',
    'difficulty': 'basica',
    'exerciseType': 'completar_codigo',
    'datasetSize': 'pequeno',
}

response = requests.post(API_URL, json=payload, timeout=120)
print('status:', response.status_code)
data = response.json()
data


In [ ]:
required_keys = {
    'title',
    'instructions',
    'starterCode',
    'solutionCode',
    'expectedOutput',
    'hints',
    'files',
    'steps',
    'acceptanceCriteria',
}

exercise = data.get('exercise', {})
missing = sorted(list(required_keys - set(exercise.keys())))
print('missing keys:', missing)
print('fallback:', data.get('meta', {}).get('fallback'))


In [ ]:
# Prueba rápida con varios temas
tests = [
    {'topic':'pandas','difficulty':'basica','exerciseType':'completar_codigo','datasetSize':'pequeno'},
    {'topic':'numpy','difficulty':'media','exerciseType':'corregir_errores','datasetSize':'pequeno'},
    {'topic':'analisis_datos','difficulty':'basica','exerciseType':'explicar_resultado','datasetSize':'mediano'},
    {'topic':'ml_basico','difficulty':'media','exerciseType':'completar_codigo','datasetSize':'mediano'},
]

fallbacks = 0
for t in tests:
    r = requests.post(API_URL, json=t, timeout=120)
    j = r.json()
    fb = j.get('meta', {}).get('fallback')
    fallbacks += 1 if fb else 0
    print(t['topic'], 'status', r.status_code, 'fallback', fb)

print('fallback rate:', fallbacks, '/', len(tests))
